<a href="https://colab.research.google.com/github/RichardTwit/App_Project_SpringBoard/blob/main/HPQ_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:

import pickle
# Load the saved model from file
with open('/content/drive/MyDrive/HPQ_Model.pkl', 'rb') as file:
    saved_model = pickle.load(file)



In [ ]:
df = pd.read_csv("/content/drive/MyDrive/RTX.csv")
df.head()

,date,volume,open,high,low,close,adjclose
0,2020-06-19,19144400,68.360001,68.360001,64.470001,64.889999,64.889999
1,2020-06-18,6431100,65.830002,67.430000,65.610001,66.879997,66.879997
2,2020-06-17,9289800,68.000000,68.199997,66.629997,66.900002,66.900002
3,2020-06-16,11523900,69.620003,70.099998,66.050003,67.820000,67.820000
4,2020-06-15,13553100,63.000000,67.760002,62.439999,67.050003,67.050003


In [ ]:

df = pd.read_csv("/content/drive/MyDrive/RTX.csv")
# Calculate Support and Resistance Levels
df['Support_Level'] = df['low'].rolling(window=20).min()

# Identify the swing highs and swing lows
df['Swing_High'] = df['high'].rolling(window=3).max()
df['Swing_Low'] = df['low'].rolling(window=3).min()

# Calculate Fibonacci levels
df['Fib_Retracement_0.382'] = df['Swing_High'] - 0.382 * (df['Swing_High'] - df['Swing_Low'])
df['Fib_Retracement_0.618'] = df['Swing_High'] - 0.618 * (df['Swing_High'] - df['Swing_Low'])

# Calculate an exponential moving average (EMA) with a span of 14 periods
df['EMA_14'] = df['close'].ewm(span=14, adjust=False).mean()

columns_to_normalize = ['volume', 'open', 'high', 'low', 'close', 'Support_Level', 'Swing_High',
       'Fib_Retracement_0.382', 'Fib_Retracement_0.618', 'EMA_14']


df[columns_to_normalize] = (df[columns_to_normalize] - df[columns_to_normalize].min()) / (df[columns_to_normalize].max() - df[columns_to_normalize].min())

#df.sort_values('date', inplace=True)


df['Price_Change'] = df['adjclose'].pct_change()

df = df.dropna()
X = df.drop(['date','Price_Change', 'adjclose', 'Swing_Low'], axis=1)
y = df['Price_Change']



In [ ]:
df

,date,volume,open,high,low,close,adjclose,Support_Level,Swing_High,Swing_Low,Fib_Retracement_0.382,Fib_Retracement_0.618,EMA_14,Price_Change
19,2020-05-22,0.102402,0.604788,0.607694,0.604195,0.606203,60.000000,0.630690,0.680393,59.330002,0.652778,0.635481,0.679233,-0.069335
20,2020-05-21,0.162532,0.583707,0.602053,0.592607,0.607221,60.099998,0.618595,0.655808,58.200001,0.633069,0.618826,0.671676,0.001667
21,2020-05-20,0.163085,0.587842,0.590568,0.587378,0.583815,57.799999,0.613136,0.607646,57.689999,0.601072,0.596954,0.661925,-0.038270
22,2020-05-19,0.227936,0.579572,0.601347,0.584404,0.585240,57.939999,0.610032,0.602003,57.400002,0.596429,0.592938,0.653669,0.002422
23,2020-05-18,0.311881,0.556978,0.586034,0.565126,0.584833,57.900002,0.589909,0.601298,55.520000,0.588692,0.580797,0.646459,-0.000690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12724,1970-01-08,0.001932,0.003496,0.003368,0.003731,0.003477,0.141476,0.002316,0.003270,0.771908,0.003450,0.003563,0.002623,-0.006329
12725,1970-01-07,0.002496,0.003595,0.003393,0.003731,0.003527,0.142377,0.002316,0.003270,0.771908,0.003450,0.003563,0.002714,0.006370
12726,1970-01-06,0.006438,0.003620,0.003418,0.003832,0.003627,0.144179,0.002316,0.003294,0.771908,0.003465,0.003573,0.002806,0.012659
12727,1970-01-05,0.003702,0.003645,0.003492,0.003882,0.003652,0.144630,0.002316,0.003369,0.771908,0.003512,0.003601,0.002889,0.003124


In [ ]:
predictions = saved_model.predict(X)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
# Calculate the MSE and R-squared score using the predicted values
mse = mean_squared_error(y, predictions)
r2 = r2_score(y, predictions)

# Print the results

print("R-squared Score:", r2)
print("Mean Squared Error:", mse)

R-squared Score: -1.9254306008126516
Mean Squared Error: 0.0009253992849108817


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DIS.csv")
df.head()

,date,volume,open,high,low,close,adjclose
0,2020-07-02,10305900,115.000000,115.099998,112.000000,112.180000,112.180000
1,2020-07-01,11930900,112.820000,115.599998,112.290001,113.010002,113.010002
2,2020-06-30,10565900,111.500000,112.050003,109.930000,111.510002,111.510002
3,2020-06-29,12584300,109.000000,111.570000,108.099998,111.519997,111.519997
4,2020-06-26,15270900,110.949997,111.199997,108.019997,109.099998,109.099998


In [ ]:

df = pd.read_csv("/content/drive/MyDrive/RTX.csv")
# Calculate Support and Resistance Levels
df['Support_Level'] = df['low'].rolling(window=20).min()

# Identify the swing highs and swing lows
df['Swing_High'] = df['high'].rolling(window=3).max()
df['Swing_Low'] = df['low'].rolling(window=3).min()

# Calculate Fibonacci levels
df['Fib_Retracement_0.382'] = df['Swing_High'] - 0.382 * (df['Swing_High'] - df['Swing_Low'])
df['Fib_Retracement_0.618'] = df['Swing_High'] - 0.618 * (df['Swing_High'] - df['Swing_Low'])

# Calculate an exponential moving average (EMA) with a span of 14 periods
df['EMA_14'] = df['close'].ewm(span=14, adjust=False).mean()

columns_to_normalize = ['volume', 'open', 'high', 'low', 'close', 'Support_Level', 'Swing_High',
       'Fib_Retracement_0.382', 'Fib_Retracement_0.618', 'EMA_14']


df[columns_to_normalize] = (df[columns_to_normalize] - df[columns_to_normalize].min()) / (df[columns_to_normalize].max() - df[columns_to_normalize].min())

#df.sort_values('date', inplace=True)


df['Price_Change'] = df['adjclose'].pct_change()

df = df.dropna()
X = df.drop(['date','Price_Change', 'adjclose', 'Swing_Low'], axis=1)
y = df['Price_Change']



In [ ]:
predictions = saved_model.predict(X)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
# Calculate the MSE and R-squared score using the predicted values
mse = mean_squared_error(y, predictions)
r2 = r2_score(y, predictions)

# Print the results

print("R-squared Score:", r2)
print("Mean Squared Error:", mse)

R-squared Score: -1.9254306008126516
Mean Squared Error: 0.0009253992849108817


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DIS.csv")
df.head()

,date,volume,open,high,low,close,adjclose
0,2020-07-02,10305900,115.000000,115.099998,112.000000,112.180000,112.180000
1,2020-07-01,11930900,112.820000,115.599998,112.290001,113.010002,113.010002
2,2020-06-30,10565900,111.500000,112.050003,109.930000,111.510002,111.510002
3,2020-06-29,12584300,109.000000,111.570000,108.099998,111.519997,111.519997
4,2020-06-26,15270900,110.949997,111.199997,108.019997,109.099998,109.099998


In [ ]:

df = pd.read_csv("/content/drive/MyDrive/Download Data - STOCK_US_XNYS_HPQ.csv")

df = df.dropna()

# Create a dictionary mapping old column names to new column names
column_mapping = {'Low': 'low',
                  'High': 'high',
                  'Open': 'open',
                  'Close' : 'close',
                  'Volume' :'volume',
                  'Date' : 'date'}

# Use the 'rename()' function to replace the column names
df = df.rename(columns=column_mapping)

df['volume']=  df['volume'].str.replace(',','')
df['volume']=  df['volume'].astype('float64')


# Calculate Support and Resistance Levels
df['Support_Level'] = df['low'].rolling(window=20).min()

# Identify the swing highs and swing lows
df['Swing_High'] = df['high'].rolling(window=3).max()
df['Swing_Low'] = df['low'].rolling(window=3).min()

# Calculate Fibonacci levels
df['Fib_Retracement_0.382'] = df['Swing_High'] - 0.382 * (df['Swing_High'] - df['Swing_Low'])
df['Fib_Retracement_0.618'] = df['Swing_High'] - 0.618 * (df['Swing_High'] - df['Swing_Low'])

# Calculate an exponential moving average (EMA) with a span of 14 periods
df['EMA_14'] = df['close'].ewm(span=14, adjust=False).mean()

columns_to_normalize = ['volume', 'open', 'high', 'low', 'close', 'Support_Level', 'Swing_High',
       'Fib_Retracement_0.382', 'Fib_Retracement_0.618', 'EMA_14']


df[columns_to_normalize] = (df[columns_to_normalize] - df[columns_to_normalize].min()) / (df[columns_to_normalize].max() - df[columns_to_normalize].min())

#df.sort_values('date', inplace=True)


df['Price_Change'] = df['close'].pct_change()


df = df.dropna()


X = df.drop(['date','Price_Change', 'Swing_Low'], axis=1)
X = X.reindex(columns=columns_to_normalize)
y = df['Price_Change']



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232 entries, 19 to 250
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   232 non-null    object 
 1   open                   232 non-null    float64
 2   high                   232 non-null    float64
 3   low                    232 non-null    float64
 4   close                  232 non-null    float64
 5   volume                 232 non-null    float64
 6   Support_Level          232 non-null    float64
 7   Swing_High             232 non-null    float64
 8   Swing_Low              232 non-null    float64
 9   Fib_Retracement_0.382  232 non-null    float64
 10  Fib_Retracement_0.618  232 non-null    float64
 11  EMA_14                 232 non-null    float64
 12  Price_Change           232 non-null    float64
dtypes: float64(12), object(1)
memory usage: 25.4+ KB


In [ ]:
predictions = saved_model.predict(X)

In [ ]:
X

,volume,open,high,low,close,Support_Level,Swing_High,Fib_Retracement_0.382,Fib_Retracement_0.618,EMA_14
19,0.188650,0.600575,0.533981,0.507843,0.436433,0.624843,0.513185,0.498436,0.489066,0.588748
20,0.187138,0.510536,0.560194,0.533333,0.555977,0.624843,0.540568,0.515652,0.499823,0.596578
21,0.336964,0.486590,0.478641,0.463725,0.448767,0.593476,0.540568,0.505799,0.483709,0.583862
22,0.755639,0.534483,0.509709,0.455882,0.414611,0.583438,0.540568,0.502646,0.478553,0.566629
23,0.195773,0.711686,0.649515,0.664706,0.592030,0.583438,0.633874,0.561309,0.515206,0.583966
...,...,...,...,...,...,...,...,...,...,...
246,0.065222,0.705939,0.661165,0.705882,0.636622,0.775408,0.657201,0.646132,0.639100,0.836474
247,0.085235,0.740421,0.719417,0.733333,0.703985,0.775408,0.706897,0.699054,0.694072,0.838196
248,0.097378,0.714559,0.677670,0.723529,0.672676,0.775408,0.706897,0.707726,0.708252,0.833994
249,0.133016,0.704023,0.648544,0.654902,0.614801,0.775408,0.706897,0.687230,0.674736,0.819824


In [ ]:
predictions

array([-3.17822263e-01,  1.16074620e-01, -9.54507293e-02, -2.48081645e-01,
       -1.65666641e-01, -1.94788780e-02, -1.47682619e-02, -1.62717685e-01,
       -6.84972015e-02, -4.16614565e-02, -1.47063570e-01,  5.26824789e-03,
       -3.77444012e-02, -9.99568719e-02,  3.86413775e-02, -2.28799654e-01,
       -1.74203075e-01, -2.26386769e-01, -4.95812118e-02, -1.01740002e-01,
       -1.96613249e-02, -1.53011852e-01, -1.57386384e-01, -1.40765667e-01,
       -7.29558028e-02, -1.16200617e-01,  4.74548003e-02, -1.45045481e-01,
       -1.19786190e-01, -1.05821850e-01, -1.36158626e-01, -1.34418796e-01,
       -1.43716687e-01, -5.56879012e-02, -3.79614904e-02, -9.70095099e-02,
       -6.17454220e-02, -1.92200755e-01, -1.03987981e-01,  1.63299228e-02,
       -1.11506687e-01, -9.90492653e-02, -1.80910091e-01,  1.37587220e-02,
       -1.97016299e-02, -5.41510692e-02, -6.93464219e-02, -1.97633904e-01,
       -1.77252969e-01, -1.26611632e-01, -1.98453285e-01, -3.43080957e-01,
       -1.22655038e-01, -

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
# Calculate the MSE and R-squared score using the predicted values
mse = mean_squared_error(y, predictions)
r2 = r2_score(y, predictions)

# Print the results

print("R-squared Score:", r2)
print("Mean Squared Error:", mse)

ValueError: ignored

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/HPQ.csv")
df.head()

,date,volume,open,high,low,close,adjclose
0,2020-07-02,10157300,17.240000,17.600000,16.99,17.070000,17.070000
1,2020-07-01,11761700,17.410000,17.590000,16.92,17.000000,17.000000
2,2020-06-30,12307100,17.110001,17.469999,16.98,17.430000,17.430000
3,2020-06-29,12255900,16.700001,17.280001,16.48,17.129999,17.129999
4,2020-06-26,30227400,16.290001,16.760000,16.08,16.610001,16.610001


In [ ]:

df = pd.read_csv("/content/drive/MyDrive/HPQ.csv")
# Calculate Support and Resistance Levels
df['Support_Level'] = df['low'].rolling(window=20).min()

# Identify the swing highs and swing lows
df['Swing_High'] = df['high'].rolling(window=3).max()
df['Swing_Low'] = df['low'].rolling(window=3).min()

# Calculate Fibonacci levels
df['Fib_Retracement_0.382'] = df['Swing_High'] - 0.382 * (df['Swing_High'] - df['Swing_Low'])
df['Fib_Retracement_0.618'] = df['Swing_High'] - 0.618 * (df['Swing_High'] - df['Swing_Low'])

# Calculate an exponential moving average (EMA) with a span of 14 periods
df['EMA_14'] = df['close'].ewm(span=14, adjust=False).mean()

columns_to_normalize = ['volume', 'open', 'high', 'low', 'close', 'Support_Level', 'Swing_High',
       'Fib_Retracement_0.382', 'Fib_Retracement_0.618', 'EMA_14']


df[columns_to_normalize] = (df[columns_to_normalize] - df[columns_to_normalize].min()) / (df[columns_to_normalize].max() - df[columns_to_normalize].min())

#df.sort_values('date', inplace=True)


df['Price_Change'] = df['adjclose'].pct_change()

df = df.dropna()
X = df.drop(['date','Price_Change', 'adjclose', 'Swing_Low'], axis=1)
y = df['Price_Change']



In [ ]:
predictions = saved_model.predict(X)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
# Calculate the MSE and R-squared score using the predicted values
mse = mean_squared_error(y, predictions)
r2 = r2_score(y, predictions)

# Print the results

print("R-squared Score:", r2)
print("Mean Squared Error:", mse)

R-squared Score: -0.6646279070083267
Mean Squared Error: 0.0008969315637373527
